In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Lasso
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split

In [2]:
crunch_companies_df = pd.read_csv("data/crunchbase-subscription-data/companies-11-2-2018_more_columns.csv")
print("Number of records: {}".format(crunch_companies_df.shape[0]))
print("Number of columns: {}".format(crunch_companies_df.shape[1]))
crunch_companies_df.head(5)

Number of records: 1000
Number of columns: 9


,Organization Name,Organization Name URL,Categories,Headquarters Location,Description,CB Rank (Company),Operating Status,Founded Date,Founded Date Precision
0,Netflix,https://www.crunchbase.com/organization/netflix,"Digital Entertainment, Media and Entertainment...","Los Gatos, California, United States",Netflix is an online platform that enables use...,1,Active,1997-01-01,day
1,Ant Financial,https://www.crunchbase.com/organization/alipay,"Banking, E-Commerce, Financial Services, FinTe...","Hangzhou, Zhejiang, China",Ant Financial is an online payment services pr...,2,Active,2014-10-01,month
2,Xiaomi,https://www.crunchbase.com/organization/xiaomi,"Consumer Electronics, Internet, Mobile","Beijing, Beijing, China",Xiaomi is an electronics and software company ...,3,Active,2010-04-04,day
3,Techstars,https://www.crunchbase.com/organization/techstars,"Finance, Financial Services, Venture Capital","Boulder, Colorado, United States",Techstars is the worldwide network that helps ...,4,Active,2006-10-01,day
4,Tesla,https://www.crunchbase.com/organization/tesla-...,"Automotive, Electronics, Innovation Management","Palo Alto, California, United States",Tesla Motors accelerates the transition to ele...,5,Active,2003-01-01,day


In [44]:
unique_cats2 = set([x for cat in [str(cats).split(', ') for cats in crunch_companies_df['Categories']] for x in cat])
len(unique_cats2)

486

In [3]:
crunch_df = pd.read_csv("data/crunchbase-data/companies.csv")
print("Number of records: {}".format(crunch_df.shape[0]))
print("Number of columns: {}".format(crunch_df.shape[1]))
crunch_df.head(5)

Number of records: 66368
Number of columns: 14


,permalink,name,homepage_url,category_list,funding_total_usd,status,country_code,state_code,region,city,funding_rounds,founded_at,first_funding_at,last_funding_at
0,/organization/-fame,#fame,http://livfame.com,Media,10000000,operating,IND,16,Mumbai,Mumbai,1,NaN,2015-01-05,2015-01-05
1,/organization/-qounter,:Qounter,http://www.qounter.com,Application Platforms|Real Time|Social Network...,700000,operating,USA,DE,DE - Other,Delaware City,2,2014-09-04,2014-03-01,2014-10-14
2,/organization/-the-one-of-them-inc-,"(THE) ONE of THEM,Inc.",http://oneofthem.jp,Apps|Games|Mobile,3406878,operating,NaN,NaN,NaN,NaN,1,NaN,2014-01-30,2014-01-30
3,/organization/0-6-com,0-6.com,http://www.0-6.com,Curated Web,2000000,operating,CHN,22,Beijing,Beijing,1,2007-01-01,2008-03-19,2008-03-19
4,/organization/004-technologies,004 Technologies,http://004gmbh.de/en/004-interact,Software,-,operating,USA,IL,"Springfield, Illinois",Champaign,1,2010-01-01,2014-07-24,2014-07-24


In [5]:
value_df = pd.read_csv("data/startup_valuations.csv", encoding='unicode_escape')
print("Number of records: {}".format(value_df.shape[0]))
print("Number of columns: {}".format(value_df.shape[1]))
value_df.head(5)

Number of records: 1469
Number of columns: 6


,Full Name,Primary Company,Seed Valuation,A Valuation,B Valuation,Valuation Increase
0,Chad Hurley,YouTube,0,"14,000,000.00","88,400,000.00",6.314285714
1,Gwyneth Paltrow,Goop Inc.,0,"40,000,000.00","65,000,000.00",1.625
2,Jason Calacanis,Inside.com,"$7,000,000.00","11,250,000.00",0,#VALUE!
3,Tony Fadell,Nest Labs,"$2,700,000.00","49,210,000.00","288,120,000.00",5.854907539
4,Matt Mullenweg,Automattic,0,"8,650,000.00","238,590,000.00",27.58265896


In [6]:
crunch_df.dropna(subset=['funding_total_usd'], inplace=True)
crunch_df.drop(crunch_df[crunch_df['country_code'] != 'USA'].index, inplace=True)

In [7]:
crunch_df.drop(columns=['permalink', 'homepage_url', 'region'], inplace=True)

In [8]:
crunch_df.shape

(37601, 11)

In [10]:
crunch_df.reset_index(drop=True, inplace=True)
crunch_df.head()

,name,category_list,funding_total_usd,status,country_code,state_code,city,funding_rounds,founded_at,first_funding_at,last_funding_at
0,:Qounter,Application Platforms|Real Time|Social Network...,700000,operating,USA,DE,Delaware City,2,2014-09-04,2014-03-01,2014-10-14
1,004 Technologies,Software,-,operating,USA,IL,Champaign,1,2010-01-01,2014-07-24,2014-07-24
2,H2O.ai,Analytics,33600000,operating,USA,CA,Mountain View,4,2011-01-01,2013-01-03,2015-11-09
3,One Inc.,Mobile,1150050,operating,USA,CA,San Francisco,3,2011-08-01,2011-07-20,2014-02-05
4,1-4 All,Entertainment|Games|Software,-,operating,USA,NC,Connellys Springs,1,NaN,2013-04-21,2013-04-21


In [11]:
index_drop_list = []
for i in crunch_df.index:
    if crunch_df['funding_total_usd'][i] == '-':
        index_drop_list.append(i)
df = crunch_df.drop(index=index_drop_list)
df.reset_index(drop=True, inplace=True)
df.head()

,name,category_list,funding_total_usd,status,country_code,state_code,city,funding_rounds,founded_at,first_funding_at,last_funding_at
0,:Qounter,Application Platforms|Real Time|Social Network...,700000,operating,USA,DE,Delaware City,2,2014-09-04,2014-03-01,2014-10-14
1,H2O.ai,Analytics,33600000,operating,USA,CA,Mountain View,4,2011-01-01,2013-01-03,2015-11-09
2,One Inc.,Mobile,1150050,operating,USA,CA,San Francisco,3,2011-08-01,2011-07-20,2014-02-05
3,1-800-DOCTORS,Health and Wellness,1750000,operating,USA,NJ,Iselin,1,1984-01-01,2011-03-02,2011-03-02
4,"1-800-PublicRelations, Inc.",Internet Marketing|Media|Public Relations,6000000,operating,USA,NY,New York,1,2013-10-24,2015-02-01,2015-02-01


In [12]:
df.shape

(31429, 11)

In [14]:
df['category_list'].astype(str)

0        Application Platforms|Real Time|Social Network...
1                                                Analytics
2                                                   Mobile
3                                      Health and Wellness
4                Internet Marketing|Media|Public Relations
                               ...                        
31424    Cloud Computing|Mobile Advertising|Video|Video...
31425                                Public Transportation
31426                                           E-Commerce
31427                                        Biotechnology
31428    Consumer Electronics|Internet of Things|Teleco...
Name: category_list, Length: 31429, dtype: object

In [16]:
unique_cats = set([x for cat in [str(cats).split('|') for cats in df['category_list']] for x in cat])
unique_cats

{'3D',
 '3D Printing',
 '3D Technology',
 'Accounting',
 'Active Lifestyle',
 'Ad Targeting',
 'Adaptive Equipment',
 'Advanced Materials',
 'Adventure Travel',
 'Advertising',
 'Advertising Exchanges',
 'Advertising Networks',
 'Advertising Platforms',
 'Advice',
 'Aerospace',
 'Agriculture',
 'Air Pollution Control',
 'Algorithms',
 'All Markets',
 'All Students',
 'Alternative Medicine',
 'Alumni',
 'Analytics',
 'Android',
 'Angels',
 'Animal Feed',
 'Anything Capital Intensive',
 'App Discovery',
 'App Marketing',
 'App Stores',
 'Application Performance Monitoring',
 'Application Platforms',
 'Apps',
 'Aquaculture',
 'Architecture',
 'Archiving',
 'Art',
 'Artificial Intelligence',
 'Artists Globally',
 'Assisitive Technology',
 'Assisted Living',
 'Auctions',
 'Audio',
 'Audiobooks',
 'Augmented Reality',
 'Auto',
 'Automated Kiosk',
 'Automotive',
 'B2B',
 'B2B Express Delivery',
 'BPO Services',
 'Babies',
 'Baby Accessories',
 'Baby Boomers',
 'Baby Safety',
 'Banking',
 'Bat

In [17]:
len(unique_cats)

835

In [18]:
df = df.join(df.category_list.str.get_dummies('|'))

In [19]:
df

,name,category_list,funding_total_usd,status,country_code,state_code,city,funding_rounds,founded_at,first_funding_at,...,Wine And Spirits,Wireless,Women,Writers,Young Adults,iOS,iPad,iPhone,iPod Touch,mHealth
0,:Qounter,Application Platforms|Real Time|Social Network...,700000,operating,USA,DE,Delaware City,2,2014-09-04,2014-03-01,...,0,0,0,0,0,0,0,0,0,0
1,H2O.ai,Analytics,33600000,operating,USA,CA,Mountain View,4,2011-01-01,2013-01-03,...,0,0,0,0,0,0,0,0,0,0
2,One Inc.,Mobile,1150050,operating,USA,CA,San Francisco,3,2011-08-01,2011-07-20,...,0,0,0,0,0,0,0,0,0,0
3,1-800-DOCTORS,Health and Wellness,1750000,operating,USA,NJ,Iselin,1,1984-01-01,2011-03-02,...,0,0,0,0,0,0,0,0,0,0
4,"1-800-PublicRelations, Inc.",Internet Marketing|Media|Public Relations,6000000,operating,USA,NY,New York,1,2013-10-24,2015-02-01,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31424,Zype,Cloud Computing|Mobile Advertising|Video|Video...,3300000,operating,USA,NY,New York,2,2013-01-01,2015-04-01,...,0,0,0,0,0,0,0,0,0,0
31425,Whisk (formerly Zypsee),Public Transportation,2150000,operating,USA,NY,New York,2,2011-01-01,2013-07-26,...,0,0,0,0,0,0,0,0,0,0
31426,Zyrra,E-Commerce,1510500,operating,USA,MA,Cambridge,4,NaN,2010-10-21,...,0,0,0,0,0,0,0,0,0,0
31427,Zystor,Biotechnology,8500000,acquired,USA,WI,Milwaukee,1,NaN,2004-12-16,...,0,0,0,0,0,0,0,0,0,0


In [22]:
df.funding_total_usd.astype(float)

0          700000.0
1        33600000.0
2         1150050.0
3         1750000.0
4         6000000.0
            ...    
31424     3300000.0
31425     2150000.0
31426     1510500.0
31427     8500000.0
31428       18192.0
Name: funding_total_usd, Length: 31429, dtype: float64

In [23]:
y = df.funding_total_usd

In [24]:
X = df.drop(columns=['name', 'category_list', 'funding_total_usd', 'status', 'country_code',
                     'state_code', 'city', 'funding_rounds', 'founded_at', 'first_funding_at', 'last_funding_at'])

In [36]:
alpha=0.9 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [37]:
lr = LinearRegression()
lr.fit(X_train, y_train)
y_hat_test = lr.predict(X_test)
r2_score(y_test, y_hat_test)

-2.6346975835429862e+23

In [38]:
lasso = Lasso(alpha = alpha, max_iter= 10000)
lasso.fit(X_train, y_train)
y_hat_lasso_test = lasso.predict(X_test)
r2_score(y_test, y_hat_lasso_test)

-0.13725113904923303

In [39]:
coef_list = []
for feature, coef in zip(X_train.columns, lasso.coef_):
    print(feature, coef)
    coef_list.append((coef, feature))

3D -7022457.041784169
3D Printing -3972729.3658264005
3D Technology -341091.50151148177
Accounting 6218107.163505247
Active Lifestyle 66817.5253280807
Ad Targeting 28008012.5411746
Adaptive Equipment 5126289.24942825
Advanced Materials -61983030.69496527
Adventure Travel 3157185.337634605
Advertising 8976664.689267598
Advertising Exchanges 4993115.898783258
Advertising Networks -5824728.342795138
Advertising Platforms -3691364.9062481862
Advice 10711758.082635436
Aerospace 16727429.433392089
Agriculture -1988633.627268614
Air Pollution Control -8977042.994604941
Algorithms -20237247.232938357
All Markets -27046914.46538037
All Students 68775307.0081531
Alternative Medicine -3455962.1919617835
Alumni 3274931.113749466
Analytics 5879258.818776922
Android -6844613.341204378
Angels 10787847.09535634
Animal Feed -19741500.283400215
Anything Capital Intensive -20840639.96393959
App Discovery 11546586.660295853
App Marketing 3701044.4717084533
App Stores -4844199.175503244
Application Perform

In [40]:
for coef in sorted(coef_list, reverse=True):
    print(coef)

(1122561783.58715, 'Limousines')
(731527020.2140285, 'Communications Infrastructure')
(650982510.8623662, 'Concentrated Solar Power')
(335718336.2767333, 'Space Travel')
(241488958.58139277, 'Film Production')
(185360460.41704124, 'Local Commerce')
(164929428.11608955, 'Wind')
(159612403.24136737, 'Coworking')
(129495609.7572111, 'Natural Gas Uses')
(127173163.83233017, 'Oil and Gas')
(125136001.47270016, 'Infrastructure Builders')
(123332664.75865498, 'Broadcasting')
(114083217.98765747, 'Debt Collecting')
(109262883.19161536, 'Information Technology')
(100636138.33661735, 'Cable')
(87204403.9856517, 'Service Industries')
(86799071.03255638, 'User Interface')
(86204479.87894066, 'Custom Retail')
(85970351.58736648, 'Social Bookmarking')
(83854080.5513685, 'Natural Resources')
(80532660.54943335, 'Credit')
(79303190.77673903, 'Trading')
(79197875.0178058, 'Public Transportation')
(78258073.50318956, 'Hi Tech')
(75416998.91527489, 'Semiconductors')
(75068279.34988338, 'Identity')
(72412